<center>
    <h1> ILI285 - Computación Científica I  / INF285 - Computación Científica </h1>
    <h2> Tarea 4 </h2>
    <h2> [S]cientific [C]omputing [T]eam </h2>
</center>

- **Nombre**: Marcelo Jara Almeyda
- **Rol**: 18654885-K
- **Email**: marcelo.jara.13@sansano.usm.cl

_Mayo 2017_

# Introducción

En esta tarea se estudiarán los métodos PALU y Gauss-Seidel para estudiar métodos iterativos, con lo que podremos comparar el desempeño de ambos en distintos puntos de interés. En este caso no nos interesa mucho su tiempo de procesamiento ya que se habla de matrices pequeñas, si no que nos interesa su afinidad al resultado esperado.

In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
from scipy.linalg import solve
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import axes3d
from matplotlib import cm
import matplotlib.mlab as mlab
from matplotlib.widgets import Slider, Button, RadioButtons
import matplotlib.pyplot as plt
from ipywidgets.widgets import interact
from ipywidgets import widgets
import matplotlib
import matplotlib.cm as cm

matplotlib.rcParams['xtick.direction'] = 'out'
matplotlib.rcParams['ytick.direction'] = 'out'
matplotlib.rcParams['axes.titlesize'] = 16
matplotlib.rcParams['axes.labelsize'] = 20
matplotlib.rcParams['xtick.labelsize'] = 16
matplotlib.rcParams['ytick.labelsize'] = 16

La función visual fue alterada para recibir un _callback_ llamado _solver_ de forma de poder entregar fácilmente el método sin reescribir este código.

In [2]:
def visual(n_steps_newton=2,x00=0.5,x01=2,solver=None):
    # Vector form of the initial guess
    x0=np.array([x00,x01])

    # The functions to be solved
    f1v=np.vectorize(lambda x,y: y-x**2)
    f2v=np.vectorize(lambda x,y: x**2+y**2-1)
    nf1f2v=np.vectorize(lambda x,y: np.linalg.norm(np.array([f1v(x,y),f2v(x,y)])))

    # The jacobian matrix for f1 and f2
    JM=np.vectorize(lambda x,y: np.array([[-2*x,1],[2*x,2*y]]))

    # Creating matrix to store all approximations made by the Newton's method
    xs=np.zeros((2,n_steps_newton+1))
    xs[:,0]=x0

    # Applying Newton's method n_steps_newton
    for n in np.arange(n_steps_newton):
        b=np.array([[f1v(xs[0,n],xs[1,n])],[f2v(xs[0,n],xs[1,n])]])
        # HINT: HERE IS WHERE YOU NEED TO USE PALU OR GAUSS-SEIDEL!!!
        A = JM(xs[0,n],xs[1,n])
        if solver is None:
            s = np.linalg.solve(A,b)
        else:
            s = solver(A, b, x0)
        xs[:,n+1]=xs[:,n] - s.T

    # Data for plotting
    x = np.linspace(-2.0, 2.0, 100)
    y = np.linspace(-2.0, 2.0, 100)
    [X,Y]=np.meshgrid(x,y)

    # Visualization! It is very important you learn about visualization in general.
    plt.figure(figsize=(15,7.5))

    plt.subplot(1, 2, 1)
    CS = plt.pcolor(X,Y,nf1f2v(X,Y))
    plt.contour(X, Y, f1v(X,Y), levels=[0], colors='k')
    plt.contour(X, Y, f2v(X,Y), levels=[0], colors='k')
    plt.colorbar(CS)
    plt.plot(xs[0,:],xs[1,:],'.r',markersize=5)
    plt.plot(xs[0,:],xs[1,:],'-r')
    plt.axis('tight')
    plt.xlabel('$x$')
    plt.ylabel('$y$')
    f1_at_x0=float(f1v(xs[0,-1],xs[1,-1]))
    f2_at_x0=float(f2v(xs[0,-1],xs[1,-1]))
    nf1f2v_at_x0=float(nf1f2v(xs[0,-1],xs[1,-1]))
    plt.title('Plotting $f_1=${:.16f},\n $f_2=${:.16f},\n $\|[f_1,f_2]\|_2=${:.16f}, \n $x_n=${:.16f}\n and $y_n=${:.16f}'.format(f1_at_x0,f2_at_x0,nf1f2v_at_x0,xs[0,-1],xs[1,-1]))
    
    plt.subplot(1, 2, 2)
    x_i=xs[0,:]
    y_i=xs[1,:]
    plt.semilogy(np.arange(n_steps_newton+1),np.abs(f1v(x_i,y_i)),'.',label='$|f_1|$',markersize=20)
    plt.semilogy(np.arange(n_steps_newton+1),np.abs(f2v(x_i,y_i)),'d',label='$|f_2|$',markersize=15,alpha=0.5)
    plt.semilogy(np.arange(n_steps_newton+1),np.abs(nf1f2v(x_i,y_i)),'s',label='$\|[f_1,f_2]\|$',markersize=12,alpha=0.5)
    plt.xlabel('n (Número de iteraciones)')
    plt.grid(True)
    plt.legend(loc='best')

    plt.show()

In [3]:
def visual_interact(n_steps_newton=2,x00=0.5,x01=2):
    visual(n_steps_newton=n_steps_newton, x00=x00, x01=x01)

interact(visual_interact, n_steps_newton=(1,10), x00=(-2,2,0.1), x01=(-2,2,0.1))

<function __main__.visual_interact>

<h3> PALU: </h3>
<p style="font-size:16px"><b>a) </b> Dentro de la función visual, reemplazar la <code>función np.linalg.solve</code> (destacado en comentarios) con el método <b>PALU</b> para resolver el sistema de ecuaciones, para los puntos iniciales indicados más abajo, ¿Que observaciones puede destacar del comportamiento del método  y las soluciones encontradas? ¿Existen mejoras si aumenta el número de iteraciones? ¿Cuantas iteraciones se necesitan para saturar la solución?</p>

1. **(5 puntos)** $\mathbf{x}_0=\langle1,2\rangle$
2. **(10 puntos)** $\mathbf{x}_0=\langle0,2\rangle$
3. **(15 puntos)** $\mathbf{x}_0=\langle-2,-2\rangle$

In [5]:
def PALU(A, b, x0):
    P, L, U = sp.linalg.lu(A)
    c = sp.linalg.solve_triangular(L, P @ b, lower=True)
    return sp.linalg.solve_triangular(U, c, lower=False)

def visual_palu(n_steps_newton=2,x00=0.5,x01=2):
    visual(n_steps_newton=n_steps_newton, x00=x00, x01=x01, solver=PALU)

interact(visual_palu, n_steps_newton=(1,20), x00=(-2,2,0.1), x01=(-2,2,0.1))

<function __main__.visual_palu>

Analizando cada punto, en $\langle1,2\rangle$ encontramos que converge sin problemas en pocas iteraciones, incluso en 3 iteraciones encontramos una respuesta bastante cercana, pero a la 6ta iteración ya la saturamos (como se aprecia en el gráfico de la derecha). Para el siguiente punto $\langle0,2\rangle$ el método falla con `singular matrix: resolution failed at diagonal 0`, indicandonos claramente que la matriz Jacobiana es singular, por lo que la descomposición no es posible. Finalmente para el punto $\langle-2,-2\rangle$ el método diverge sin poder encontrar una solución. Esto se puede deber a que el método converge localmente y nuestro _initial guess_ se encontraba muy lejano, aumentar la cantidad de iteraciones no soluciona el problema en este caso.

<h3> Métodos Iterativos: </h3>
<p style="font-size:16px"><b>b)</b> (1) Acople el método iterativo de <b>Gauss-Seidel</b> al visualizador (llamelo visual2), (2) incluya un cuarto parametro al widget para controlar el número de iteraciones de <b>Gauss-Seidel</b>, (3) use el algoritmo implementado para los siguientes puntos iniciales indicados más abajo. En ambos casos ¿Como afecta el número de iteraciones en la convergencia de la solución? </p>

1. **(30 puntos)** Construir visual2: (1) Acoplando Gauss-Seidel y (2) Incluyendo un cuarto parámetro al widget
2. **(10 puntos)** $\mathbf{x}_0=\langle1,2\rangle$
3. **(10 puntos)** $\mathbf{x}_0=\langle-2,-2\rangle$

In [7]:
def gauss_seidel(A, b, x0, steps=5):
    D = np.diag(np.diag(A))
    L = np.tril(A) - D
    U = np.triu(A) - D

    sol = x0
    for i in range(steps):
        sol = np.linalg.inv(L + D) @ (b - (U @ sol).reshape((2,1)))
    return sol

def gauss_seidel_factory(steps=5):
    return lambda A, b, x0: gauss_seidel(A, b, x0, steps=steps)

def visual_gs(n_steps_newton=2, n_gs_steps=2, x00=1, x01=2):
    visual(n_steps_newton=n_steps_newton, x00=x00, x01=x01, solver=gauss_seidel_factory(n_gs_steps))

interact(visual_gs, n_steps_newton=(1,20), n_gs_steps=(1,50), x00=(-2,2,0.1), x01=(-2,2,0.1))

<function __main__.visual_gs>

Usando este método tenemos dos dependencias de iteraciones, Newton y Gauss-Seidel. Para el punto $\langle1,2\rangle$ usaremos inicialmente 50 iteraciones GS (lo que nos da un punto base para poder analizar Newton), la solución no es muy precisa para una cantidad de iteraciones relativamente alta, como se puede ver en el gráfico de la derecha, el error está en el vecindario de $10^{-4}$ y se satura en la 4ta iteración.

Para iteraciones menores de GS el método tiende a diverger constántemente (digo tiende ya que no diverge para cada valor de iteraciones bajo).

Para el punto $\langle-2,-2\rangle$ el método diverge sin importar la cantidad de iteraciones, como se ve en el gráfico de la derecha los puntos se ven dispersos, sin demostrar un patrón de convergencia.

<p style="font-size:16px"><b>c) (20 Puntos)</b> Respecto al análisis realizado en el punto anterior, ¿Que información se desprende del segundo gráfico? ¿Que medida corresponde la curva $\|[f_1 , f_2]\|_2$ ? </p>

El segundo gráfico nos deja saber cómo converge nuestro método, se usó para hacer las conclusiones de divergencia y de saturación. La curva corresponde a la norma del error usando las funciones $f_1$ y $f_2$, evaluandolas en cada punto de cada iteración.

# Conclusión

En esta tarea se pudo trabajar con los métodos Gauss-Seidel y PALU más cercanamente, siendo aplicados para analizar su convergencia en distintos puntos conflictivos. Se aprende que Gauss-Seidel es bastante dependiente de las iteraciones, pero podría servir mucho mejor para entregar aproximaciones rápidas, a su vez PALU es preciso, pero no sirve para entregar aproximaciones ya que es un método _atómico_.

En general para estos casos no hay gran diferencia entre usar ambos métodos, pero si necesitara usar alguno para estos calculos usaría PALU ya que me fue más sencillo interpretas y usar sus soluciones. 

# Referencias

- https://docs.scipy.org/doc/